In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import uuid

from datasets import Dataset

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
url = "https://www.kinomania.ru/film/441560/script/262"
response = requests.get(url=url)
soup = BeautifulSoup(response.text)
movie_description = soup.find("div", class_="scripts-info-caption")
print(movie_description.text)
movie_script = soup.find("div", class_="row-text")
print(movie_script.text)
title = soup.find("h1", class_="pagetitle").text
print(title)

'Чужой против Хищника'

In [4]:
class KinomaniaScriptsParser:
    def __init__(self) -> None:
        self.urls = []
        self.dataset = []

    def parse_scripts(self):
        self.urls = self.parse_scripts_links()

        for url in tqdm(self.urls):
            response = requests.get(url=url)
            soup = BeautifulSoup(response.text)

            movie_description = soup.find("div", class_="scripts-info-caption").text
            movie_script = soup.find("div", class_="row-text").text
            title = soup.find("h1", class_="pagetitle").text

            self.dataset.append(
                {
                    "title": title,
                    "movie_description": movie_description,
                    "movie_script": movie_script,
                }
            )

    def parse_scripts_links(self):
        url = "https://www.kinomania.ru/scripts"
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text)
        all_scripts = soup.find_all("div", class_="list-tile-item")
        all_links = []
        for item in all_scripts:
            script_href = item.find("a")["href"]
            script_href = f"https://www.kinomania.ru/{script_href}"
            all_links.append(script_href)

        return all_links


parser = KinomaniaScriptsParser()
parser.parse_scripts()

100%|██████████| 27/27 [00:14<00:00,  1.92it/s]


In [11]:
url = "https://www.kinomania.ru/scripts"
response = requests.get(url=url)
soup = BeautifulSoup(response.text)
all_scripts = soup.find_all("div", class_="list-tile-item")

In [15]:
script_href = all_scripts[0].find("a")["href"]
script_href = f"https://www.kinomania.ru/{script_href}"
script_href

'https://www.kinomania.ru//film/401988/script/268'

### convert to dataset


In [43]:
def remove_new_lines(string):
    string = re.sub(r"[\n\t\r]{3,}", "\n\n", string)
    string = re.sub(r"[ ]{2,}", " ", string)
    return string


dataset = parser.dataset

# text = remove_new_lines(dataset[1]['movie_script'])
for i in range(len(dataset)):
    dataset[i]["movie_script"] = remove_new_lines(dataset[i]["movie_script"])

In [5]:
pd.DataFrame(data=dataset).to_csv(
    "./verbalist/datasets/kinomania_scripts/kinomania_scripts.csv",
    index=False,
)

In [10]:
dataset = pd.read_csv("./verbalist/datasets/kinomania_scripts/kinomania_scripts.csv")
dataset = dataset.to_dict('records')
dataset = {item['title']: item for item in dataset}

dataset2 = pd.read_csv("./verbalist/datasets/kinomania_scripts/kinomania_scripts_translated_summary.csv")
dataset2 = dataset2.to_dict('records')
for item in dataset2:
    dataset[item['title']]['movie_description'] = item['movie_description']

dataset = list(dataset.values())

In [14]:
dataset = Dataset.from_list(dataset)
dataset

Dataset({
    features: ['movie_script', 'movie_description', 'title'],
    num_rows: 27
})

In [15]:
dataset.push_to_hub("dim/kinomania_scripts")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:02<00:00,  2.37s/it]


In [31]:
from datasets import load_dataset

dataset = load_dataset("dim/kinomania_scripts")
dataset = dataset["train"]
dataset = dataset.to_list()
dataset[0]

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 457.39it/s]


Dataset parquet downloaded and prepared to /home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--kinomania_scripts-3eae84281497b8df/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 875.09it/s]


{'movie_script': '\nИЗ ЗАТЕМНЕНИЯ:\n\nЭКСТ. ЮЖНЫЙ БОСТОН. ПАРАД В ДЕНЬ СВЯТОГО ПАТРИКА -- ДЕНЬ\n\n ПЕРЕХОД:\nИНТ. БАР ЭЛЬ СТРИТ, ЮЖНЫЙ БОСТОН -- ВЕЧЕР\n\nБар грязный, более чем просто неубран. Даже если здесь работает повар, его сейчас нет \nна месте. По мере того, как камера панорамирует поперек нескольких пустых столиков, мы \nпочти слышим запах вчерашнего пива и рассыпанной закуски на полу. \n\n ЧАКИ\n О, Боже, у меня для вас ребята просто \n самая офигенная история.\n \nПо мере того как камера поднимается, мы находим ЧЕТЫРЕХ МОЛОДЫХ ЛЮДЕЙ, сидящих \nвокруг стола прямо позади барной стойки.\n\n ВСЕ (хором) \n О нет. Опять начинается.\n\nПарень, которому предоставлено слово, - ЧАКИ САЛЛИВАН, 20, самый здоровый в компании. \nОн громок, неистов, прирожденный развлекатель. Вслед за ним сидит УИЛЛ ХАНТИНГ, 20, \nсимпатичный и уверенный в себе, негласный лидер. Справа от Уилла БИЛЛИ МАКБРАЙД, 22, \nтяжелый, тихий, тот, с кем вы определенно не захотели бы ругаться. И наконец \nМОРГАН ОМАЙЛ

In [2]:
import openai
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

openai.api_key = open("./chat_gpt_token").read()

In [3]:
def chat_with_chatgpt(prompt, model="gpt-3.5-turbo"):
    chat_completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )

    return chat_completion["choices"][0]["message"]["content"].strip()


user_prompt = "Hello world."
chatbot_response = chat_with_chatgpt(user_prompt)
print(chatbot_response)

Hello! How can I assist you today?


In [11]:
num = 4
full_review = dataset[num]["movie_description"]

input_text = f"""
Summary: {full_review}

Imagine you are user who wants to write a film script.
Write prompt in one or maximum two sentences and use language from section 'Summary' (english or russian).
write a prompt based on 'Summary' that asks to write a film script. Answer only prompt. Don't mention and use any part information from 'Summary' section.
Translate prompt to language used from 'Summary'
"""
print(input_text)
chatbot_response = chat_with_chatgpt(input_text, model='gpt-4')
print("=" * 100)
print("=" * 100)
print(chatbot_response)


Summary: Маленький калифорнийский городок Санта - Роза терроризирует маньяк - убийца. Маньяк - киноман, и свои дьявольские идеи он черпает в кинофильмах. Избранные им молодые жертвы могут избежать гибели, если только они следуют правилам, которые предписывают ленты, которых он насмотрелся: не открывать дверь незнакомцам и не спрашивать `кто там?`, не прятаться в кладовой, не отвечать на телефонные звонки, не пить и не употреблять наркотики, не заниматься сексом. И самое главное - никогда, ни при каких обстоятельствах не кричать! 

Imagine you are user who wants to write a film script.
Write prompt in one or maximum two sentences and use language from section 'Summary' (english or russian).
write a prompt based on 'Summary' that asks to write a film script. Answer only prompt. Don't mention and use any part information from 'Summary' section.
Translate prompt to language used from 'Summary'

Write a film script about a small Californian town terrorized by a maniac killer who gets his i

In [26]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

for i in range(len(dataset)):
    full_review = dataset[i]['movie_description']
    language = detect(full_review)
    input_text = f"""
    Short Plot: {full_review}

    Imagine you are user who wants to write a film script.
    Write prompt in one or maximum two sentences and use language from section 'Short Plot' (write prompt in {language} language).
    write a prompt based on 'Short Plot' that asks to write a film script. Answer only prompt. Don't summarize in prompt from 'Short Plot' section.
    """
    print(input_text)
    chatbot_response = chat_with_chatgpt(input_text, model='gpt-4')
    dataset[i]['prompt'] = chatbot_response
    print(chatbot_response) 
    print("="*100)
    # break


    Short Plot: Уилл Хантинг - двадцатилетний парень из Бостона, который то и дело ввязывается в неприятные истории. Неожиданно он открывает в себе феноменальные математические способности. И когда полиция арестовывает его за очередную драку, профессор математики берет его под свою опеку, но при одном условии: Уилл должен пройти курс психотерапии. Сеансы `перевоспитания`, начавшиеся с недоверия, постепенно перерастают в дружбу между Уиллом и его наставником. 

    Imagine you are user who wants to write a film script.
    Write prompt in one or maximum two sentences and use language from section 'Short Plot' (write prompt in ru language).
    write a prompt based on 'Short Plot' that asks to write a film script. Answer only prompt. Don't summarize in prompt from 'Short Plot' section.
    
Напишите сценарий фильма, в котором двадцатилетний парень из Бостона обнаруживает свои феноменальные математические способности и под руководством профессора математики проходит курс психотерапии, чт

In [30]:
from datasets import Dataset
Dataset.from_list(dataset).push_to_hub("dim/kinomania_scripts")

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  4.77it/s]
Updating downloaded metadata with the new split.
